# 1D heat equation

In [1]:
#include<run_hpx.cpp>
#include<vector>
#include<fstream>
#include <iostream>

IncrementalExecutor::executeFunction: symbol '__emutls_v._ZSt11__once_call' unresolved while linking function '__cxx_global_var_initcling_module_1_.28'!
IncrementalExecutor::executeFunction: symbol '__emutls_v._ZSt15__once_callable' unresolved while linking function '__cxx_global_var_initcling_module_1_.28'!


## Method to save the out to a file for plotitng the result

In [2]:
void saveFile(std::vector<hpx::shared_future<double>> U){

  std::ofstream myfile;
  myfile.open("out.txt");
  
  for( auto u : U)
  myfile <<  u.get() << std::endl;
  myfile.close();

}

## Default parameters

In [3]:
double k = 0.5;     // heat transfer coefficient
double dt = 1.;     // time step
double dx = 1.;     // grid spacing
size_t nx = 100;
size_t nt = 45;

## Function to map the indicies 

In [4]:
inline std::size_t idx(std::size_t i, int dir, std::size_t size)
{
    if(i == 0 && dir == -1)
        return size-1;
    if(i == size-1 && dir == +1)
        return 0;

    HPX_ASSERT((i + dir) < size);

    return i + dir;
}

## Simulation class

In [5]:
class stepper {
    
    
    public:
    
    // We use a shared future to be able to copy the partion
    typedef hpx::shared_future<double> partition;

    // Our data for one time step
    typedef std::vector<partition> space;
    
     // do all the work on 'nx' data points for 'nt' time steps
    space do_work(std::size_t nx, std::size_t nt)
    {
        // U[t][i] is the state of position i at time t.
        std::vector<space> U(2);
        for (space& s : U)
            s.resize(nx);

        // Initial conditions: f(0, i) = i
        // Note we fill the intial conditions with futures, since the 
        // the partion is now a shared future object
        for (std::size_t i = 0; i != nx; ++i)
            U[0][i] = hpx::make_ready_future(double(i));
        
        // We need to unwrap the heat function, since we pass future<double> and
        // not double values anymore to it
        auto Op = hpx::util::unwrapping(&stepper::heat);

        // Actual time step loop
        for (std::size_t t = 0; t != nt; ++t)
        {
            space const& current = U[t % 2];
            space& next = U[(t + 1) % 2];

             for (std::size_t i = 0; i != nx; ++i)
            {
                next[i] = hpx::dataflow(
                        hpx::launch::async, Op,
                        current[idx(i, -1, nx)], current[i], current[idx(i, +1, nx)]
                    );
            }
           
        }

        // Return the solution at time-step 'nt'.
        return U[nt % 2];
    }
    
    
    private:
    
    // Our operator
    static double heat(double left, double middle, double right)
    {
        return middle + (k*dt/(dx*dx)) * (left - 2*middle + right);
    }    
    
};

## Start timer

In [6]:
stepper step;

In [7]:
run_hpx([](){


std::uint64_t t = hpx::util::high_resolution_clock::now();

stepper::space solution = step.do_work(nx, nt);


std::uint64_t elapsed = hpx::util::high_resolution_clock::now() - t;
std::uint64_t const os_thread_count = hpx::get_os_thread_count();

std::cout << "Computation took " << t << " on " << os_thread_count << " threads" << std::endl;
    
std::cout << "Output written to out.txt!" << std::endl;
saveFile(solution);

});

Computation took 8495716119653563 on 4 threads
Output written to out.txt!


(void) @0x7f3f9ca9fbe8
